In [41]:
import pandas as pd

In [43]:
import os
gpx_files_all = os.listdir('data_1/run/')
gpx_files = []
for each in gpx_files_all:
    if 'timestamp_updated' in each:
        gpx_files.append('data_1/run/' + each)

gpx_files

['data_1/run/9085281003_timestamp_updated.gpx',
 'data_1/run/9073431927_timestamp_updated.gpx',
 'data_1/run/9012800883_timestamp_updated.gpx',
 'data_1/run/9097682164_timestamp_updated.gpx',
 'data_1/run/9031086524_timestamp_updated.gpx',
 'data_1/run/9079039783_timestamp_updated.gpx',
 'data_1/run/9043324564_timestamp_updated.gpx',
 'data_1/run/9055381881_timestamp_updated.gpx',
 'data_1/run/8987485568_timestamp_updated.gpx',
 'data_1/run/9073432175_timestamp_updated.gpx',
 'data_1/run/9000082336_timestamp_updated.gpx',
 'data_1/run/8993941962_timestamp_updated.gpx',
 'data_1/run/9091217140_timestamp_updated.gpx',
 'data_1/run/9067051054_timestamp_updated.gpx',
 'data_1/run/9006640377_timestamp_updated.gpx',
 'data_1/run/9060798722_timestamp_updated.gpx',
 'data_1/run/9037047109_timestamp_updated.gpx',
 'data_1/run/9049516744_timestamp_updated.gpx',
 'data_1/run/9018082038_timestamp_updated.gpx']

In [44]:
from xml.etree import ElementTree as ET

# Register the namespace for GPX files
ET.register_namespace("", "http://www.topografix.com/GPX/1/1")

def extract_start_end_points(gpx_file):
    # Parse the GPX file
    tree = ET.parse(gpx_file)
    root = tree.getroot()

    # Get all trackpoints
    trkpts = root.findall(".//{http://www.topografix.com/GPX/1/1}trkpt")

    # Extract coordinates for the first (start) and last (end) trackpoint
    start_point = (float(trkpts[0].attrib['lat']), float(trkpts[0].attrib['lon']))
    end_point = (float(trkpts[-1].attrib['lat']), float(trkpts[-1].attrib['lon']))

    return start_point, end_point

# List to store start and end points for all activities
start_end_points = []


In [45]:
# For each GPX file, extract start and end points and append to the list
for gpx_file in gpx_files:
    start_end_points.append(extract_start_end_points(gpx_file))

print(start_end_points)

[((12.328704, 76.611763), (12.328684, 76.611732)), ((12.328659, 76.611737), (12.328556, 76.611839)), ((12.328665, 76.611782), (12.327271, 76.612383)), ((12.328669, 76.611809), (12.325719, 76.612935)), ((12.328642, 76.611763), (12.328434, 76.611932)), ((12.328659, 76.61177), (12.328253, 76.611949)), ((12.328643, 76.611694), (12.307739, 76.636752)), ((12.328769, 76.611748), (12.328682, 76.611762)), ((12.328601, 76.611765), (12.328682, 76.611733)), ((12.327219, 76.612646), (12.327178, 76.612696)), ((12.32864, 76.611786), (12.327032, 76.612504)), ((12.328634, 76.611726), (12.328679, 76.61174)), ((12.328719, 76.611786), (12.329168, 76.611465)), ((12.328718, 76.611742), (12.31387, 76.627622)), ((12.328618, 76.611816), (12.327406, 76.612326)), ((12.2884, 76.685164), (12.288, 76.683714)), ((12.328608, 76.611742), (12.326958, 76.612387)), ((12.328637, 76.611686), (12.327437, 76.612332)), ((12.328633, 76.61181), (12.322294, 76.616515))]


In [46]:
start_end_points

[((12.328704, 76.611763), (12.328684, 76.611732)),
 ((12.328659, 76.611737), (12.328556, 76.611839)),
 ((12.328665, 76.611782), (12.327271, 76.612383)),
 ((12.328669, 76.611809), (12.325719, 76.612935)),
 ((12.328642, 76.611763), (12.328434, 76.611932)),
 ((12.328659, 76.61177), (12.328253, 76.611949)),
 ((12.328643, 76.611694), (12.307739, 76.636752)),
 ((12.328769, 76.611748), (12.328682, 76.611762)),
 ((12.328601, 76.611765), (12.328682, 76.611733)),
 ((12.327219, 76.612646), (12.327178, 76.612696)),
 ((12.32864, 76.611786), (12.327032, 76.612504)),
 ((12.328634, 76.611726), (12.328679, 76.61174)),
 ((12.328719, 76.611786), (12.329168, 76.611465)),
 ((12.328718, 76.611742), (12.31387, 76.627622)),
 ((12.328618, 76.611816), (12.327406, 76.612326)),
 ((12.2884, 76.685164), (12.288, 76.683714)),
 ((12.328608, 76.611742), (12.326958, 76.612387)),
 ((12.328637, 76.611686), (12.327437, 76.612332)),
 ((12.328633, 76.61181), (12.322294, 76.616515))]

In [47]:
from sklearn.cluster import DBSCAN
import numpy as np

# We have to flat the list of tuples into a list of points
points = [point for start_end in start_end_points for point in start_end]

# DBSCAN requires 2D array-like input
X = np.array(points)

# Define the parameters for DBSCAN
eps_in_radians = 0.25 / 6371.0088  # this is approximately 500 meters in radians
min_samples = 6  # adjust this number based on how many activities need to share a location for it to be considered common

# Run DBSCAN
db = DBSCAN(eps=eps_in_radians, min_samples=min_samples, metric='haversine').fit(np.radians(X))

# Get the cluster labels
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)

print('Estimated number of clusters: %d' % n_clusters)


Estimated number of clusters: 1


In [48]:
import folium

# Create a map centered at the mean of the points
map_center = [np.mean(X[:, 0]), np.mean(X[:, 1])]
m = folium.Map(location=map_center, zoom_start=13)

# Create a color dictionary for each cluster ID
colors = {i: '#%02X%02X%02X' % tuple(np.random.choice(range(256), size=3)) for i in range(n_clusters)}

# add a marker for every record in the filtered data, use a clustered view
for idx, coordinates in enumerate(X):
    cluster_id = labels[idx]
    if cluster_id != -1:  # Exclude the noise points
        folium.CircleMarker(
            location=coordinates,
            fill=True,
            radius=5,
            weight=1,
            color=colors[cluster_id],
            fill_color=colors[cluster_id],
            fill_opacity=1
        ).add_to(m)

m.save('cluster_map.html')


In [49]:
m

In [51]:
from xml.etree import ElementTree as ET
from datetime import datetime
import os

def get_start_end_points_timestamps(gpx_folder):
    start_end_points_timestamps = []

    for gpx_file in os.listdir(gpx_folder):
        if gpx_file.endswith(".gpx"):
            tree = ET.parse(os.path.join(gpx_folder, gpx_file))
            root = tree.getroot()

            # Extract all trackpoints (trkpt elements)
            trackpoints = root.findall('.//{http://www.topografix.com/GPX/1/1}trkpt')
            if trackpoints:
                # Get the first (start) and last (end) trackpoint
                start_point, end_point = trackpoints[0], trackpoints[-1]

                # Extract lat, lon and time from start and end points
                start_lat, start_lon = float(start_point.attrib['lat']), float(start_point.attrib['lon'])
                end_lat, end_lon = float(end_point.attrib['lat']), float(end_point.attrib['lon'])

                start_time_text = start_point.find('{http://www.topografix.com/GPX/1/1}time').text
                end_time_text = end_point.find('{http://www.topografix.com/GPX/1/1}time').text

                start_time = datetime.fromisoformat(start_time_text.replace('Z', '+00:00'))
                end_time = datetime.fromisoformat(end_time_text.replace('Z', '+00:00'))

                # Append the start and end coordinates and corresponding timestamps to the list
                start_end_points_timestamps.append((start_lat, start_lon, start_time, end_lat, end_lon, end_time))

    return start_end_points_timestamps

gpx_folder = "data/run/"
start_end_points_timestamps = get_start_end_points_timestamps(gpx_folder)


In [52]:
start_end_points_timestamps

[(33.643497,
  -117.843057,
  datetime.datetime(2023, 5, 5, 0, 57, 45, tzinfo=datetime.timezone.utc),
  33.642964,
  -117.844376,
  datetime.datetime(2023, 5, 5, 2, 10, 59, tzinfo=datetime.timezone.utc)),
 (33.668187,
  -117.827028,
  datetime.datetime(2023, 5, 17, 2, 41, 50, tzinfo=datetime.timezone.utc),
  33.6692,
  -117.82781,
  datetime.datetime(2023, 5, 17, 3, 34, 8, tzinfo=datetime.timezone.utc)),
 (33.668103,
  -117.827115,
  datetime.datetime(2023, 5, 6, 2, 52, 40, tzinfo=datetime.timezone.utc),
  33.67046,
  -117.830735,
  datetime.datetime(2023, 5, 6, 3, 40, 51, tzinfo=datetime.timezone.utc)),
 (33.668242,
  -117.826983,
  datetime.datetime(2023, 5, 10, 2, 16, 51, tzinfo=datetime.timezone.utc),
  33.668434,
  -117.827186,
  datetime.datetime(2023, 5, 10, 3, 21, 15, tzinfo=datetime.timezone.utc)),
 (33.668252,
  -117.826971,
  datetime.datetime(2023, 5, 11, 14, 59, 8, tzinfo=datetime.timezone.utc),
  33.668828,
  -117.827632,
  datetime.datetime(2023, 5, 11, 16, 9, 17, tzinfo

In [54]:
from sklearn.preprocessing import StandardScaler
import numpy as np

# Separate coordinates and timestamps
coordinates = [(start_lat, start_lon, end_lat, end_lon) for start_lat, start_lon, start_time, end_lat, end_lon, end_time in start_end_points_timestamps]
timestamps = [(start_time.timestamp(), end_time.timestamp()) for start_lat, start_lon, start_time, end_lat, end_lon, end_time in start_end_points_timestamps]

# Flatten lists
coordinates = [coordinate for sublist in coordinates for coordinate in sublist]
timestamps = [timestamp for sublist in timestamps for timestamp in sublist]

# Reshape into 2D arrays
coordinates = np.array(coordinates).reshape(-1, 2)
timestamps = np.array(timestamps).reshape(-1, 1)

# Scale timestamps
scaler = StandardScaler()
timestamps_scaled = scaler.fit_transform(timestamps)

# Merge back into one array
X_scaled = np.column_stack((coordinates, timestamps_scaled))


In [55]:
# Define the parameters for DBSCAN
eps_in_radians = 0.5 / 6371.0088  # this is approximately 500 meters in radians
min_samples = 2  # adjust this number based on how many activities need to share a location for it to be considered common

# Run DBSCAN
db = DBSCAN(eps=eps_in_radians, min_samples=min_samples).fit(X_scaled) # Note we are not specifying the metric, so Euclidean distance will be used

# Get the cluster labels
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)

print('Estimated number of clusters: %d' % n_clusters)


Estimated number of clusters: 0


In [57]:
timestamps

array([[1.68324826e+09],
       [1.68325266e+09],
       [1.68429131e+09],
       [1.68429445e+09],
       [1.68334156e+09],
       [1.68334445e+09],
       [1.68368501e+09],
       [1.68368888e+09],
       [1.68381715e+09],
       [1.68382136e+09],
       [1.68398868e+09],
       [1.68399589e+09],
       [1.68454826e+09],
       [1.68455266e+09],
       [1.68437842e+09],
       [1.68438263e+09],
       [1.68350768e+09],
       [1.68351175e+09]])